In [1]:
from time_series_transform.transform_core_api.time_series_transformer import Time_Series_Transformer
from time_series_transform.stock_transform.stock_transfromer import Stock_Transformer
import pandas as pd
import numpy as np
import pandas_ta as ta

In [2]:
def up_or_down(current,lead):
    if current < lead:
        return 'up'
    elif current > lead:
        return "down"
    else:
        return "unchange"
def up_down_transform(data):
    currentList = data['Close_GOOGL']
    leadList = data['Close_GOOGL_lead_1']
    res = []
    for c,l in zip(currentList,leadList):
        res.append(up_or_down(c,l))
    return res

In [3]:
strategy = ta.Strategy(
    name= 'mystrategy',
    ta=[
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 7},
        {"kind": "ema", "length": 20},
        {"kind": "bbands", "length": 20},
        {"kind": "bbands", "length": 50},
        {"kind": "bbands", "length": 30},
        {"kind": "rsi","prefix":"rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
    ]
)

In [4]:
st = Stock_Transformer.from_stock_engine_period(["GOOGL","NDAQ","GOLD"],'3y','yahoo')
st = st.get_technial_indicator(strategy)
st = st.dropna()
df = st.to_pandas(expandCategory= True, expandTime = False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.8s finished


In [5]:
tst = Time_Series_Transformer.from_pandas(df,'Date',None)
tst = tst.make_lead('Close_GOOGL',1,'_lead_')
tst = tst.transform(['Close_GOOGL','Close_GOOGL_lead_1'],'change',up_down_transform)
tst = tst.make_label("change")
X,y = tst.to_pandas(sepLabel = True)

# Sklearn

In [6]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV,TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from time_series_transform.sklearn.transformer import Lag_Transformer

In [10]:
testX,testY = X.tail(60),y.tail(60)
trainX,trainY = X.drop(testX.index),y.drop(testY.index)

In [44]:
pip = Pipeline(
    [
        ("lag",Lag_Transformer(list(range(1,20)),time_col = 'Date')),
        ("impute",SimpleImputer(strategy = 'median')),
        ("pca",PCA()),
        ('rf',RandomForestClassifier())
     ]
)

In [45]:
searchParam = {
    "lag__lag_nums":[list(range(1,20)),list(range(1,50)),list(range(1,100))],
#     "svm__C":np.arange(0.01,1,0.01),
    "pca__n_components":list(range(2,30)),
    "rf__n_estimators":list(range(100,500)),
#     "rf__min_samples_split":list(range(2,20))
}
randPip = RandomizedSearchCV(
    pip,
    searchParam,
    cv= TimeSeriesSplit(5),
    n_iter=30,
    n_jobs = 5
)

In [46]:
randPip = randPip.fit(trainX,trainY)
prd = randPip.predict(testX)

C:\Users\Allen Chiang\anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [47]:
print(classification_report(testY,prd))

              precision    recall  f1-score   support

        down       0.53      0.64      0.58        25
    unchange       0.00      0.00      0.00         1
          up       0.70      0.62      0.66        34

    accuracy                           0.62        60
   macro avg       0.41      0.42      0.41        60
weighted avg       0.62      0.62      0.61        60



C:\Users\Allen Chiang\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [48]:
confusion_matrix(testY,prd)

array([[16,  0,  9],
       [ 1,  0,  0],
       [13,  0, 21]], dtype=int64)

In [49]:
prd

array(['up', 'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up',
       'up', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down',
       'down', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down',
       'down', 'down', 'down', 'up', 'up', 'up', 'down', 'down', 'down',
       'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'down', 'down',
       'down', 'down', 'down', 'down', 'down', 'down', 'down', 'down',
       'down', 'down', 'down', 'down'], dtype=object)

# Signal

In [50]:
def buy_sell_signal(dataList):
    res = []
    tmp ='down'
    for i in dataList:
        if tmp == i or i == 'unchange':
            if i == 'unchange':
                res.append('hold')
                continue
            tmp = i
            res.append('hold')
        else:
            tmp = i
            res.append(i)
    return res

In [51]:
det = pd.DataFrame({'prd':buy_sell_signal(prd)})
det['Close'] = testX.Close_GOOGL.tolist()
det['Date'] = testX.Date.tolist()

In [52]:
det['Buy'] = det[['Close','prd']].apply(lambda x: x.Close if x.prd == 'up' else 0,axis =1)
det['Sell'] = det[['Close','prd']].apply(lambda x: x.Close if x.prd == 'down' else 0,axis =1)
buySignal = det[det.Buy > 0][['Date','Buy']]
sellSignal = det[det.Sell > 0][['Date','Sell']]

In [53]:
det_tst = Time_Series_Transformer.from_pandas(det,'Date',None)
det_tst.plot(["Close"],'info').add_marker(
    x = buySignal['Date'],
    y = buySignal['Buy'],color = 'green',
    legendName = 'Buy').add_marker(
    x = sellSignal['Date'],
    y = sellSignal['Sell'],color = 'red',
    legendName = 'Sell')

In [55]:
det.Sell.sum() - det.Buy.sum()

286.2701416015625

# Intra-Day Trade

In [7]:
strategy = ta.Strategy(
    name= 'mystrategy',
    ta=[
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 7},
        {"kind": "ema", "length": 20},
        {"kind": "ema", "length": 3},
        {"kind": "bbands", "length": 20},
        {"kind": "bbands", "length": 50},
        {"kind": "bbands", "length": 30},
        {"kind": "rsi","prefix":"rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
    ]
)

In [8]:
st = Stock_Transformer.from_stock_engine_intraday(["GOOGL","NDAQ","GOLD"],'2020-12-27','2020-12-31','yahoo',interval='30m')
st = st.get_technial_indicator(strategy)
st = st.dropna()
df = st.to_pandas(expandCategory= True, expandTime = False,preprocessType = 'remove')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


IndexError: iloc cannot enlarge its target object

In [120]:
tst = Time_Series_Transformer.from_pandas(df,'Datetime',None)
tst = tst.make_lead('Close_GOOGL',1,'_lead_')
tst = tst.transform(['Close_GOOGL','Close_GOOGL_lead_1'],'change',up_down_transform)
tst = tst.make_label("change")
X,y = tst.to_pandas(sepLabel = True)

In [121]:
testX,testY = X.tail(500),y.tail(500)
trainX,trainY = X.drop(testX.index),y.drop(testY.index)

In [122]:
pip = Pipeline(
    [
        ("lag",Lag_Transformer(list(range(1,20)),time_col = 'Datetime')),
        ("impute",SimpleImputer(strategy = 'median')),
        ("pca",PCA()),
        ('rf',RandomForestClassifier())
     ]
)

In [123]:
searchParam = {
    "lag__lag_nums":[list(range(1,20)),list(range(1,50)),list(range(1,100))],
    "pca__n_components":list(range(2,30)),
    "rf__n_estimators":list(range(100,500)),
}
randPip = RandomizedSearchCV(
    pip,
    searchParam,
    cv= TimeSeriesSplit(5),
    n_iter=30,
    n_jobs = 5
)

In [124]:
randPip = randPip.fit(trainX,trainY)
prd = randPip.predict(testX)

C:\Users\Allen Chiang\anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [125]:
print(classification_report(testY,prd))

              precision    recall  f1-score   support

        down       0.55      0.49      0.52       264
    unchange       0.00      0.00      0.00         2
          up       0.48      0.55      0.51       234

    accuracy                           0.51       500
   macro avg       0.34      0.35      0.34       500
weighted avg       0.52      0.51      0.51       500



C:\Users\Allen Chiang\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [126]:
confusion_matrix(testY,prd)

array([[129,   0, 135],
       [  0,   0,   2],
       [106,   0, 128]], dtype=int64)

In [127]:
det = pd.DataFrame({'prd':buy_sell_signal(prd)})
det['Close'] = testX.Close_GOOGL.tolist()
det['Datetime'] = testX.Datetime.tolist()

In [128]:
det['Buy'] = det[['Close','prd']].apply(lambda x: x.Close if x.prd == 'up' else 0,axis =1)
det['Sell'] = det[['Close','prd']].apply(lambda x: x.Close if x.prd == 'down' else 0,axis =1)
buySignal = det[det.Buy > 0][['Datetime','Buy']]
sellSignal = det[det.Sell > 0][['Datetime','Sell']]

In [129]:
det_tst = Time_Series_Transformer.from_pandas(det,'Datetime',None)
det_tst.plot(["Close"],'info').add_marker(
    x = buySignal['Datetime'],
    y = buySignal['Buy'],color = 'green',
    legendName = 'Buy').add_marker(
    x = sellSignal['Datetime'],
    y = sellSignal['Sell'],color = 'red',
    legendName = 'Sell')

In [130]:
det.Sell.sum() - det.Buy.sum()

-1752.760986328125

In [132]:
# !pip install fix_yahoo_finance -U

In [4]:
Stock_Transformer.from_stock_engine_intraday(["GOOGL",'AAPL'],'2020-12-27','2020-12-31','yahoo',interval='30m').to_pandas()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,symbol
0,2020-12-28 14:30:00,1744.910034,1755.060059,1741.819946,1753.890015,134200,0,0,GOOGL
1,2020-12-28 15:00:00,1755.000000,1757.849976,1749.849976,1756.989990,73854,0,0,GOOGL
2,2020-12-28 15:30:00,1757.000000,1767.430054,1755.800049,1762.844971,82034,0,0,GOOGL
3,2020-12-28 16:00:00,1762.280029,1765.307617,1762.010010,1763.479980,48326,0,0,GOOGL
4,2020-12-28 16:30:00,1763.569946,1767.000000,1763.045044,1766.530029,52059,0,0,GOOGL
...,...,...,...,...,...,...,...,...,...
79,2020-12-30 20:00:00,133.764999,133.850006,133.399994,133.703400,5632752,0,0,AAPL
80,2020-12-30 20:30:00,133.699997,134.179993,133.490005,133.679993,9694328,0,0,AAPL
81,2020-12-31 14:30:00,134.080002,134.679993,133.199997,133.434998,13548575,0,0,AAPL
82,2020-12-31 15:00:00,133.440002,133.600006,132.650299,132.884995,9513875,0,0,AAPL
